# SVD prediction

In [1]:
# serve python 3.11.9
import os 
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName('SVD Big') \
    .master('yarn') \
    .config('spark.driver.memory', '4g') \
    .config('spark.executor.instances', '3') \
    .config('spark.executor.memory', '6g') \
    .config("spark.locality.wait.node", "0") \
    .config("spark.sql.pivotMaxValues", "99999") \
    .getOrCreate()
    # driver non serve così grande 
    # pivot max values 
    # .config('spark.executor.instances', '2') \
    #.config('spark.executor.memoryOverhead', '2g') \
    #.config("spark.hadoop.fs.defaultFS", "hdfs://localhost:50054") \


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/10 17:13:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/10 17:14:00 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [5]:
# Test the spark session
print(f"Spark Version: {spark.version}")

Spark Version: 3.5.4


In [6]:
spark.sparkContext._conf.getAll()  # check the config

[('spark.app.submitTime', '1736333609136'),
 ('spark.app.startTime', '1736333609501'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'),
 ('spark.driver.

In [3]:
hdfs_port = 9000

In [4]:
import pandas as pd
import numpy as np

ratings = spark.read.csv('hdfs://master:'+ str(hdfs_port) +'/dataset/ratings.csv', header=True, inferSchema=True)
#ratings = spark.read.csv('hdfs://master:'+ str(hdfs_port) +'/dataset/ratingsBig.csv', header=True, inferSchema=True)
ratings.limit(5).toPandas()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
ratings = ratings.drop('timestamp')

In [6]:
ratings.limit(5).toPandas()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [6]:
num_users = ratings.select("userId").distinct().count()
num_movies = ratings.select("movieId").distinct().count()

In [8]:
print(num_users)
print(num_movies)

610
9724


In [9]:
print(type(ratings))

<class 'pyspark.sql.dataframe.DataFrame'>


In [6]:
movies = spark.read.csv('hdfs://master:'+ str(hdfs_port) +'/dataset/movies.csv', header=True, inferSchema=True)
#movies = spark.read.csv('hdfs://master:'+ str(hdfs_port) +'/dataset/moviesBig.csv', header=True, inferSchema=True)
movies.limit(5).toPandas()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
user_movie_ratings = ratings.join(movies, on='movieId', how='inner')
user_movie_ratings.limit(5).toPandas()

,movieId,userId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,3,1,4.0,Grumpier Old Men (1995),Comedy|Romance
2,6,1,4.0,Heat (1995),Action|Crime|Thriller
3,47,1,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,50,1,5.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [8]:
import time

start_time = time.time()
user_movie_ratings_pivot = user_movie_ratings.groupBy("userId").pivot("movieId").max("rating").fillna(0)
print("--- %s seconds ---" % (time.time() - start_time))

--- 11.226202487945557 seconds ---


In [9]:
user_movie_ratings_pivot.limit(5).toPandas()

25/01/10 16:35:23 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/01/10 16:35:28 WARN DAGScheduler: Broadcasting large task binary with size 2.3 MiB
25/01/10 16:35:34 WARN DAGScheduler: Broadcasting large task binary with size 7.4 MiB
                                                                                

,userId,1,2,3,4,5,6,7,8,9,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
0,463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,471,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,540,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
print(type(user_movie_ratings_pivot))

<class 'pyspark.sql.dataframe.DataFrame'>


In [9]:
from pyspark.mllib.linalg.distributed import CoordinateMatrix

temp = user_movie_ratings_pivot.rdd.map(list)

25/01/10 17:17:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/01/10 17:17:24 WARN DAGScheduler: Broadcasting large task binary with size 2.3 MiB
[Stage 20:>                                                         (0 + 1) / 1]

In [10]:
from pyspark.mllib.linalg.distributed import RowMatrix

temp_Rows = RowMatrix(temp)

In [11]:
k = 20
start_time = time.time()
svd= temp_Rows.computeSVD(k, computeU=True)
print("--- %s seconds ---" % (time.time() - start_time))

25/01/10 17:17:57 WARN DAGScheduler: Broadcasting large task binary with size 7.6 MiB
25/01/10 17:18:11 WARN RowMatrix: The input data is not directly cached, which may hurt performance if its parent RDDs are also uncached.
25/01/10 17:18:11 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.arpack.JNIARPACK
25/01/10 17:18:14 WARN DAGScheduler: Broadcasting large task binary with size 7.6 MiB
25/01/10 17:18:27 WARN DAGScheduler: Broadcasting large task binary with size 7.6 MiB
25/01/10 17:18:38 WARN DAGScheduler: Broadcasting large task binary with size 7.6 MiB
25/01/10 17:18:49 WARN DAGScheduler: Broadcasting large task binary with size 7.6 MiB
25/01/10 17:19:01 WARN DAGScheduler: Broadcasting large task binary with size 7.6 MiB
25/01/10 17:19:12 WARN DAGScheduler: Broadcasting large task binary with size 7.6 MiB
25/01/10 17:19:24 WARN DAGScheduler: Broadcasting large task binary with size 7.6 MiB
25/01/10 17:19:35 WARN DAGScheduler: Broadcasting large task bi

--- 712.7905120849609 seconds ---


25/01/10 17:29:49 WARN RowMatrix: The input data was not directly cached, which may hurt performance if its parent RDDs are also uncached.


In [ ]:
U = svd.U  # RowMatrix

num_rows_U = U.numRows() 
num_cols_U = len(s)
print("Dim matrice U:", num_rows_U,"x", num_cols_U)

In [ ]:
print(U.rows.collect())

In [ ]:
s = svd.s  # DenseVector
num_values_s = len(s)
print("Dim matrice S:", num_values_s)


In [18]:
print(s)

[8715.254040390664,430.66003946141376,230.3788472265473,181.44393394518877,163.28636239451632,153.82002599951502,146.85492016634328,134.61133915340173,122.65670927470916,121.36767523102908,113.09990478390701,109.1314684760936,107.89709023215889,105.48233406374217,101.62980571860767,99.81198283034432,99.26946681656825,97.09582747304856,93.36879143710415,92.24948301358381]


In [ ]:
V = svd.V  # DenseMatrix

num_rows_V = V.numRows  # Numero di righe di V
num_cols_V = V.numCols  # Numero di colonne di V
print("Dim matrice V:", num_rows_V, "x", num_cols_V)

In [20]:
print(V)

DenseMatrix([[ 9.99279559e-01, -3.53312555e-02,  2.53591031e-03, ...,
              -3.21291236e-04, -3.76444606e-04, -5.10174839e-04],
             [ 3.42252350e-03,  5.54252970e-02, -2.25570771e-02, ...,
              -2.43450049e-03, -1.42151584e-02, -2.81342608e-02],
             [ 1.66228674e-03,  3.37532887e-02, -1.48811343e-04, ...,
              -1.68999584e-03,  5.44739834e-03,  1.70020413e-02],
             ...,
             [ 8.48140142e-06, -2.16343807e-05,  6.23134881e-04, ...,
               7.57944951e-05,  8.16376194e-05,  9.26359228e-05],
             [ 8.48140142e-06, -2.16343807e-05,  6.23134881e-04, ...,
               7.57944951e-05,  8.16376194e-05,  9.26359228e-05],
             [ 1.74565034e-05,  1.63420109e-04,  1.31813185e-03, ...,
              -3.89118583e-04,  8.98424983e-04,  4.28757431e-04]],
            shape=(9725, 20))


In [18]:
# Check memory occupation

bytes_per_float = 8

memory_U = num_rows_U * num_cols_U * bytes_per_float
memory_S = num_values_s * bytes_per_float
memory_V = num_rows_V * num_cols_V * bytes_per_float

total_memory = memory_U + memory_S + memory_V

print(f"Memoria occupata da U: {memory_U / (1024**2):.2f} MB")
print(f"Memoria occupata da S: {memory_S / (1024**2):.2f} MB")
print(f"Memoria occupata da V: {memory_V / (1024**2):.2f} MB")
print(f"Memoria totale: {total_memory / (1024**2):.2f} MB")

Dim matrice U: 610 x 20
Dim matrice S: 20
Dim matrice V: 9725 x 20
Memoria occupata da U: 0.09 MB
Memoria occupata da S: 0.00 MB
Memoria occupata da V: 1.48 MB
Memoria totale: 1.58 MB


Data la compressione ottuneta dal calcolo di SVD (le matrici occupano qualche MB di memoria), è possibile utilizzare python per calcolare il prodotto matriciale tramite numpy e ricostruire la matrice.
Per effettuare il ranking dei film da consigliare si può utilizzare un approccio simile ad ALS, quindi filtrare la riga dell'utente scelto per i film che non ha visto e prendere i primi dieci ordinati per valore ricostruito

In [ ]:
spark.stop()